In [46]:
import pandas as pd
import numpy as np
import os
import matplotlib as mpl

In [47]:
ACPT= pd.read_csv(r'data\AdmissionsCorePopulatedTable.csv')
PCPT= pd.read_csv(r'data\PatientCorePopulatedTable.csv')
ADCPT= pd.read_csv(r'data\AdmissionsDiagnosesCorePopulatedTable.csv')
LCPT= pd.read_csv(r'data\LabsCorePopulatedTable.csv')
ACPT['AdmissionStartDate']= pd.to_datetime(ACPT['AdmissionStartDate'])
ACPT['AdmissionEndDate']= pd.to_datetime(ACPT['AdmissionEndDate'])
ACPT['Duration']=ACPT['AdmissionEndDate']-ACPT['AdmissionStartDate']

In [48]:
ACPT.sort_values(['PatientID','AdmissionID'], axis = 0, ascending = True, inplace = True, na_position ='last')
PC = ACPT.groupby(by='PatientID', as_index=False).agg({'AdmissionID': pd.Series.nunique})
ACPT=ACPT.reset_index()
start=ACPT['AdmissionStartDate']
end=ACPT['AdmissionEndDate']
start=start.reset_index()
end=end.reset_index()

In [49]:
def Insert_row(row_number, df, row_value):
    start_upper = 0
    end_upper = row_number 
    start_lower = row_number 
    end_lower = df.shape[0] 
    upper_half = [*range(start_upper, end_upper, 1)] 
    lower_half = [*range(start_lower, end_lower, 1)] 
    lower_half = [x.__add__(1) for x in lower_half]
    index_ = upper_half + lower_half 
    df.index = index_ 
    df.loc[row_number] = row_value 
    df = df.sort_index()
    return df 

In [50]:
def diff2(start, end, arr):
    s=len(start)
    e=len(end)
    arrlen=len(arr)
    k=0;
    ReAd= pd.DataFrame()
    ReAd[0]="ReAdmission Duration"
    for i in arr:
        for j in range (k,k+i):
            if(j==k+i-1):
                diff1=pd.to_timedelta('-1 days 00:00:00')
                ReAd=Insert_row(j,ReAd,diff1)
                break;
            elif(j<len(start)):
                diff1=start[j+1]-end[j]
                ReAd=Insert_row(j,ReAd,diff1)
        k=k+i;
    return ReAd

In [51]:
ReAd=diff2(start['AdmissionStartDate'],end['AdmissionEndDate'],PC['AdmissionID'])

In [52]:
ACPT['ReAdDuration']=ReAd

In [53]:
arrn=[]
for i in range (0,372):
    arrn.append(ACPT.iloc[i,6].days)
ACPT['READMISSION']=arrn

In [54]:
arrn2=[]
for i in range (0,372):
    arrn2.append(ACPT.iloc[i,5].days)
ACPT['STAYDURATION']=arrn2

In [55]:
ACPT['READ'] = [True if (x <=600 and x!=-1) else False for x in ACPT['READMISSION']] 

In [56]:
ADCPT.sort_values(['PatientID','AdmissionID'], axis = 0, ascending = True, inplace = True, na_position ='last')
PCPT.sort_values('PatientID', axis = 0, ascending = True, inplace = True, na_position ='last')
newACPT=ACPT.copy()
newADCPT=ADCPT.copy()
newACPT.reset_index()
newADCPT.reset_index()
FADCPT=pd.merge(ACPT,ADCPT,on=['PatientID','AdmissionID'])

In [57]:
FADCPT1=pd.merge(PCPT,FADCPT,on='PatientID')

In [58]:
FADCPT1['PatientDateOfBirth']= pd.to_datetime(FADCPT1['PatientDateOfBirth'])
FADCPT1['Age']=FADCPT1['AdmissionStartDate']-FADCPT1['PatientDateOfBirth']
arr=[]
for i in range(len(FADCPT1)):
    arr.append(FADCPT1.iloc[i,18].days)
arr
arr2=[]
for i in arr:
    arr2.append(i/(12*30))
arr2
FADCPT1['Age2']=arr2

In [59]:
FADCPT1['Age2']=FADCPT1['Age2'].astype(int)

In [60]:
lval={
    'URINALYSIS: RED BLOOD CELLS' :0 , 'METABOLIC: GLUCOSE' :70 , 'CBC: MCH':27,
       'METABOLIC: CALCIUM' : 8.8, 'CBC: RED BLOOD CELL COUNT':4 ,
       'URINALYSIS: PH':5 , 'METABOLIC: TOTAL PROTEIN':6.4,
       'METABOLIC: CHLORIDE':98, 'CBC: LYMPHOCYTES':1,
       'CBC: ABSOLUTE LYMPHOCYTES':20, 'METABOLIC: SODIUM':135,
       'URINALYSIS: SPECIFIC GRAVITY':1.015, 'METABOLIC: BILI TOTAL':0,
       'URINALYSIS: WHITE BLOOD CELLS':0, 'CBC: EOSINOPHILS':0.1,
       'METABOLIC: ALK PHOS':50, 'CBC: RDW':11, 'METABOLIC: AST/SGOT':15,
       'METABOLIC: CREATININE':0.6, 'CBC: NEUTROPHILS':2, 'CBC: BASOPHILS':0.0,
       'CBC: MONOCYTES':0.1, 'METABOLIC: BUN':7, 'CBC: WHITE BLOOD CELL COUNT':5,
       'CBC: PLATELET COUNT':150, 'METABOLIC: POTASSIUM':3.5,
       'METABOLIC: ANION GAP':5, 'CBC: HEMATOCRIT':35, 'CBC: HEMOGLOBIN':12,
       'METABOLIC: CARBON DIOXIDE':21, 'CBC: ABSOLUTE NEUTROPHILS':50,
       'METABOLIC: ALBUMIN':3.4, 'CBC: MCHC':32, 'CBC: MEAN CORPUSCULAR VOLUME':82,
       'METABOLIC: ALT/SGPT':20
}
uval={
    'URINALYSIS: RED BLOOD CELLS' :3 , 'METABOLIC: GLUCOSE' :109 , 'CBC: MCH':33,
       'METABOLIC: CALCIUM' : 10.5, 'CBC: RED BLOOD CELL COUNT':6 ,
       'URINALYSIS: PH':7 , 'METABOLIC: TOTAL PROTEIN':8.2,
       'METABOLIC: CHLORIDE':110, 'CBC: LYMPHOCYTES':4,
       'CBC: ABSOLUTE LYMPHOCYTES':30, 'METABOLIC: SODIUM':148,
       'URINALYSIS: SPECIFIC GRAVITY':1.025 , 'METABOLIC: BILI TOTAL':1,
       'URINALYSIS: WHITE BLOOD CELLS':5, 'CBC: EOSINOPHILS':0.5,
       'METABOLIC: ALK PHOS':136, 'CBC: RDW':15.6, 'METABOLIC: AST/SGOT':37,
       'METABOLIC: CREATININE':1, 'CBC: NEUTROPHILS':9, 'CBC: BASOPHILS':0.2,
       'CBC: MONOCYTES':1.0, 'METABOLIC: BUN':20, 'CBC: WHITE BLOOD CELL COUNT':10,
       'CBC: PLATELET COUNT':400, 'METABOLIC: POTASSIUM':5.3,
       'METABOLIC: ANION GAP':15, 'CBC: HEMATOCRIT':47, 'CBC: HEMOGLOBIN':16,
       'METABOLIC: CARBON DIOXIDE':32, 'CBC: ABSOLUTE NEUTROPHILS':75,
       'METABOLIC: ALBUMIN':5, 'CBC: MCHC':36, 'CBC: MEAN CORPUSCULAR VOLUME':100,
       'METABOLIC: ALT/SGPT':65
}

In [61]:
LCPT.sort_values(['PatientID','AdmissionID','LabName'], axis = 0, ascending = True, inplace = True, na_position ='last')

In [62]:
LCPT1=LCPT
LCPT1=LCPT1.reset_index()
LCPT1['lval']=LCPT1.LabName.map(lval)
LCPT1['uval']=LCPT1.LabName.map(uval)

In [63]:
LCPT1['IsNormal']=""

In [64]:
LCPT1.LabDateTime=pd.to_datetime(LCPT1.LabDateTime)

In [65]:
lval=LCPT1.lval
uval=LCPT1.uval
labvalue=LCPT1.LabValue

In [66]:
arr=[]
for i in LCPT1.index:
    x=lval[i]
    y=uval[i]
    z=labvalue[i]
    if(z>=x and z<=y):
        arr.append(True)
    else:
        arr.append(False)

In [67]:
LCPT1['IsNormal']=arr

In [68]:
newdf=pd.merge(FADCPT1,LCPT1,on=['PatientID','AdmissionID'])

In [69]:
#newdf

In [70]:
#newdf.to_csv('FINALEHR.csv')

In [71]:
json_list=(newdf.groupby(["PatientID", "PatientGender", 'PatientDateOfBirth', 'PatientRace','PatientMaritalStatus', 'PatientLanguage','PatientPopulationPercentageBelowPoverty', 'AdmissionID','AdmissionStartDate', 'AdmissionEndDate','READMISSION', 'STAYDURATION', 'READ', 'PrimaryDiagnosisCode','PrimaryDiagnosisDescription','Age2',"LabDateTime"])
          .apply(lambda x: x[['LabName','LabValue', 'LabUnits','IsNormal']].to_dict("r"))
          .reset_index()
          .rename(columns={0:"LabInfo"})
          .groupby(["PatientID", "PatientGender", 'PatientDateOfBirth', 'PatientRace','PatientMaritalStatus', 'PatientLanguage','PatientPopulationPercentageBelowPoverty', 'AdmissionID','AdmissionStartDate', 'AdmissionEndDate','READMISSION', 'STAYDURATION', 'READ', 'PrimaryDiagnosisCode','PrimaryDiagnosisDescription','Age2'])
          .apply(lambda x: x[["LabDateTime", "LabInfo"]].to_dict("r"))
          .reset_index()
          .rename(columns={0:"LabReports"})
          .groupby(["PatientID", "PatientGender", 'PatientDateOfBirth', 'PatientRace','PatientMaritalStatus', 'PatientLanguage','PatientPopulationPercentageBelowPoverty'])
          .apply(lambda x: x[['AdmissionID','AdmissionStartDate', 'AdmissionEndDate','READMISSION', 'STAYDURATION', 'READ', 'PrimaryDiagnosisCode','PrimaryDiagnosisDescription','Age2', "LabReports"]].to_dict("r"))
          .reset_index()
          .rename(columns={0:"AdmissionsInfo"})
          .to_json(orient="records"))

In [72]:
import json

In [73]:
j=json.loads(json_list)

In [74]:
#import sys
#sys.stdout = open('OUTPUT2.json', 'w')
#print(j)

In [75]:
#print(j)